In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import pymysql
from sklearn.preprocessing import MinMaxScaler

tf.set_random_seed(777)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
keep_prob = 0.7
nb_classes = 10
num_layer = 6
learning_rate = 0.001
training_epoch = 15
batch_size = 100
weight_init = tf.contrib.layers.xavier_initializer()

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, nb_classes])
keep_prob = tf.placeholder(tf.float32)

b_list = [784, 512, 512, 125, 125, nb_classes]
w_list = [[784, 784], [784, 512], [512, 512], [512, 125], [125, 125], [125, nb_classes]]

In [5]:
def dense(x, W, b, activation=False, dropout=False, keep_prob=1.0):
    z = tf.add(tf.matmul(x, W), b)
    if activation:
        z = tf.nn.relu(z)
    if dropout:
        z = tf.nn.dropout(z, keep_prob=keep_prob)
    return z

In [6]:
def network(x, weights, biases, num_layer, keep_prob=1.0):
    _dense = None
    for i, v in enumerate(zip(sorted(weights), sorted(biases))):
        w = v[0]
        b = v[1]
        if i == 0:
            _dense = dense(x, weights[w], biases[b], activation=True, dropout=True, keep_prob=keep_prob)
        elif i == (num_layer - 1):
            _dense = dense(_dense, weights[w], biases[b])
        else:
            _dense = dense(_dense, weights[w], biases[b], activation=True, dropout=True, keep_prob=keep_prob)
    return _dense

In [7]:
bias_dict = {'b'+str(i): tf.Variable(tf.random_normal([shapes])) for i, shapes in enumerate(b_list)}
weight_dict = {'w'+str(i): tf.get_variable('w'+str(i), shapes, initializer=weight_init) for i, shapes in enumerate(w_list)}

In [8]:
hypothesis = network(X, weights=weight_dict, biases=bias_dict, num_layer=num_layer, keep_prob=keep_prob)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            avg_cost += c/total_batch
        print('Epoch: ', '%04d'%(epoch+1), 'Cost: ', '{:.9f}'.format(avg_cost))
        
    print('Learning Finished')
    
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1}))

Epoch:  0001 Cost:  0.626879102
Epoch:  0002 Cost:  0.203292643
Epoch:  0003 Cost:  0.152090772
Epoch:  0004 Cost:  0.120794811
Epoch:  0005 Cost:  0.108736874
Epoch:  0006 Cost:  0.090146769
Epoch:  0007 Cost:  0.084605574
Epoch:  0008 Cost:  0.080198392
Epoch:  0009 Cost:  0.067030381
Epoch:  0010 Cost:  0.065307336
Epoch:  0011 Cost:  0.056555864
Epoch:  0012 Cost:  0.061134016
Epoch:  0013 Cost:  0.054387257
Epoch:  0014 Cost:  0.053089166
Epoch:  0015 Cost:  0.048949999
Learning Finished
Accuracy:  0.9835
